In [1]:
import sys
import os 
import numpy as np
import pandas as pd
from scipy.stats.qmc import Sobol
from SALib.util import scale_samples
from SALib.sample import sobol
from SALib.analyze import sobol as sobol_analyze
import random

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.sampling.sampler import get_polarisation_curve_samples, build_fixed_parameters, sample_parameters, PARAMETER_RANGES
from configuration.settings import current_density_parameters, physical_parameters, computing_parameters, operating_inputs
from modules.display_modules import plot_lambda
from model.AlphaPEM import AlphaPEM


In [87]:
PARAMETER_RANGES_hyper_box ={'Tfc':( 340.974 , 351.485266662598 ),'Pa_des':( 2e+05 , 256999.999999997 ),
                             'Sa':( 1.1 , 3 ),'Phi_a_des':( 0.31 , 1 ),'epsilon_gdl':( 0.60624999775 , 0.701 ),
                             'tau':( 1.02 , 2.689 ),'epsilon_mc':( 0.2325 , 0.4 ),'epsilon_c':( 0.245 , 0.286 ),
                             'e':( 3 , 5 ),'Re':( 5.7e-07 , 4.2125e-06 ),'i0_c_ref':( 1.3955 , 2.79 ),
                             'kappa_co':( 15.0585 , 37.821626953125 ),'kappa_c':( 0 , 1.88512650296844 ),
                             'a_slim':( 0.056 , 0.900493421521382 ),'b_slim':( 0.0550000005 , 0.542000003 ),
                             'a_switch':( 0.499999995 , 0.842 )}

In [2]:
hyperbox_80 = pd.read_csv("../data/raw//hyperbox_prim_150625.csv", sep=';')
hyperbox_dict_80 = dict(zip(hyperbox_80['parameter'], zip(hyperbox_80['lower'], hyperbox_80['upper'])))
#{row_i[0]: (row_i[1], row_i[2]) for row_i in hyperbox_80.values.tolist()}

In [3]:
PARAMETER_RANGES

{'Tfc': (333, 363),
 'Pa_des': (130000.0, 300000.0),
 'Pc_des': None,
 'Sc': (1.1, 3),
 'Phi_c_des': (0.1, 0.7),
 'epsilon_gdl': (0.55, 0.8),
 'tau': (1.0, 4.0),
 'epsilon_mc': (0.15, 0.4),
 'epsilon_c': (0.15, 0.3),
 'e': [3, 4, 5],
 'Re': (5e-07, 5e-06),
 'i0_c_ref': (0.001, 500),
 'kappa_co': (0.01, 40),
 'kappa_c': (0, 100),
 'a_slim': (0, 1),
 'b_slim': (0, 1),
 'a_switch': (0, 1)}

In [4]:

## boundaries keeping operational conditions unrestricted. The undertermined conditions are restrictied by the hyperbox

PARAMETER_RANGES_hyperbox_PRIM_2 = PARAMETER_RANGES.copy()
#PARAMETER_RANGES_hyperbox_PRIM_2.update(hyperbox_dict_80)
keys_to_filter = ['epsilon_gdl','tau', 'epsilon_mc', 'epsilon_c', 'e', 'Re', 'i0_c_ref', 'kappa_co', 'kappa_c', 'a_slim', 'b_slim', 'a_switch']
restricted_undertermined_conditions = {k: hyperbox_dict_80[k] for k in keys_to_filter if k in hyperbox_dict_80}
PARAMETER_RANGES_hyperbox_PRIM_2.update(restricted_undertermined_conditions)

In [ ]:
#PARAMETER_RANGES_hyperbox_PRIM_2['Sa']= (1.1, 3.0)
#PARAMETER_RANGES_hyperbox_PRIM_2['Phi_a_des'] = (0.1, 1.0)

In [ ]:
#PARAMETER_RANGES['kappa_c']= [0.0, 1]

In [ ]:
def make_bounds_exclusive(bounds, epsilon=1e-8):
    return [(low + epsilon, high - epsilon) for (low, high) in bounds]

def prepare_problem(PARAMETER_, dependent_param="Pc_des"):
    independent_params = {
        k: v for k, v in PARAMETER_.items()
        if k != dependent_param
    }
    # Add dummy variable to later compute dependent_param
    z_key = f"{dependent_param}"
    independent_params[z_key] = (0, 1)

    bounds = [v if isinstance(v, tuple) else [min(v), max(v)] for v in independent_params.values()]
    bounds = make_bounds_exclusive(bounds)

    problem = {
        "num_vars": len(independent_params),
        "names": list(independent_params.keys()),
        "bounds": bounds,
        "groups": list(independent_params.keys())  # optional but useful for group-based SA
    }
    return problem

problem = prepare_problem(PARAMETER_=PARAMETER_RANGES_hyperbox_PRIM_2,dependent_param = "Pc_des")


In [9]:
seed = 33
N=25
np.random.seed(seed)
random.seed(seed)
samples = sobol.sample(problem, N=N, calc_second_order= False, scramble= True, seed=seed)
pd.DataFrame(samples).to_pickle("../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted.pkl")

In [10]:
samples.shape

(475, 17)

In [11]:
samples = pd.read_pickle('../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted.pkl')
samples.columns = problem['names']
samples['Pc_des'] = samples['Pa_des']-20000
#samples['e'] = np.round(samples['e'])
samples['Sa'] = 1.3
samples['Phi_a_des'] = 0.5

In [12]:
samples.head()

,Tfc,Pa_des,Sc,Phi_c_des,epsilon_gdl,tau,epsilon_mc,epsilon_c,e,Re,i0_c_ref,kappa_co,kappa_c,a_slim,b_slim,a_switch,Pc_des,Sa,Phi_a_des
0,335.015900,215267.894967,2.784156,0.607013,0.680188,1.111181,0.286904,0.25788,4.29822,0.000002,61.274834,24.454516,1.637951,0.055778,0.915271,0.767995,195267.894967,1.3,0.5
1,335.930971,215267.894967,2.784156,0.607013,0.680188,1.111181,0.286904,0.25788,4.29822,0.000002,61.274834,24.454516,1.637951,0.055778,0.915271,0.767995,195267.894967,1.3,0.5
2,335.015900,164712.670632,2.784156,0.607013,0.680188,1.111181,0.286904,0.25788,4.29822,0.000002,61.274834,24.454516,1.637951,0.055778,0.915271,0.767995,144712.670632,1.3,0.5
3,335.015900,215267.894967,2.837283,0.607013,0.680188,1.111181,0.286904,0.25788,4.29822,0.000002,61.274834,24.454516,1.637951,0.055778,0.915271,0.767995,195267.894967,1.3,0.5
4,335.015900,215267.894967,2.784156,0.340410,0.680188,1.111181,0.286904,0.25788,4.29822,0.000002,61.274834,24.454516,1.637951,0.055778,0.915271,0.767995,195267.894967,1.3,0.5


In [13]:
pd.DataFrame(samples).to_pickle("../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted.pkl")

## We read the saved configurations and star the simulations

In [34]:
samples = pd.read_pickle('../data/raw/sobol_sampling_configurations_hyper_box_undertermined_fixed.pkl')

In [35]:
split_dejvis, split_nathaly, split_camila = np.array_split(samples, 3)

d:\Users\kju10\Documents\LMU-STATISTICS & DATA SCIENCE MASTER\WS2425\Consulting\project\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\env\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [36]:
samples.shape

(450, 19)

In [37]:
print(f'shape1 {split_dejvis.shape}, shape2 {split_nathaly.shape} and shape3 {split_camila.shape}')


shape1 (150, 19), shape2 (150, 19) and shape3 (150, 19)


In [ ]:
save_filepath = "../data/raw/sobol_sampling_split_hyperbox80_PRIM.pkl"
results = get_polarisation_curve_samples(sampled_parameters=samples.iloc[61:,:].to_dict(orient='records'), fixed_parameters = build_fixed_parameters(), save_path=save_filepath, save_every=10)#


✅ Saved 10 samples to ../data/raw/sobol_sampling_split_hyperbox80_2.pkl
✅ Saved 20 samples to ../data/raw/sobol_sampling_split_hyperbox80_2.pkl
✅ Saved 30 samples to ../data/raw/sobol_sampling_split_hyperbox80_2.pkl
✅ Saved 40 samples to ../data/raw/sobol_sampling_split_hyperbox80_2.pkl

📁 Final save complete: ../data/raw/sobol_sampling_split_hyperbox80_2.pkl with 44 valid samples.
